<a href="https://colab.research.google.com/github/cedro3/others2/blob/main/Anime2Sketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##セットアップ

In [ ]:
# githubからコードを取得
! git  clone https://github.com/Mukosame/Anime2Sketch.git
%cd Anime2Sketch

# weights フォルダー削除
import os
import shutil
if os.path.isdir('weights'):
    shutil.rmtree('weights') 

# 学習済みパラメータとサンプル動画のダウンロード
import gdown
gdown.download('https://drive.google.com/u/1/uc?id=1wIC4VXRBRgFaPzL2v8RURCCAYu2J7zsH', 'weights.zip', quiet=False)
gdown.download('https://drive.google.com/u/1/uc?id=1mTszJ5AcYW15x-AFsxdRZw-ARzypwYDK', 'movie.zip', quiet=False)
! unzip weights.zip
! unzip movie.zip

##アニメ＋スケッチ動画を作成

**アニメ動画からアニメ画とスケッチ画を作成**

In [ ]:
import os
import shutil
import cv2
 
# imagesフォルダーリセット
if os.path.isdir('images'):
    shutil.rmtree('images') 
os.makedirs('images', exist_ok=True)

# outフォルダーリセット
if os.path.isdir('out'):
    shutil.rmtree('out') 
os.makedirs('out', exist_ok=True)

# ビデオから静止画を切り出す関数 
def video_2_images(video_file= './movie/movie03.mp4', 
                   image_dir='./images/', 
                   image_file='%s.png'):
 
    # Initial setting
    i = 0
    interval = 3
    length = 300 # リミッター
    
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:    
           cv2.imwrite(image_dir+image_file % str(int(i/interval)).zfill(6), frame)
        i += 1 
    cap.release()  

# アニメ動画からアニメ画を作成 
video_2_images()

# アニメ画をスケッチ画に変換
! python3 test.py --dataroot images --load_size 512 --output_dir out

**アニメ画とスケッチ画から動画を作成**

In [ ]:
# output1.mp4 〜 output3.mp4 をリセット
import os
if os.path.exists('./output1.mp4'):
   os.remove('./output1.mp4')
if os.path.exists('./output2.mp4'):
   os.remove('./output2.mp4')
if os.path.exists('./output3.mp4'):
   os.remove('./output3.mp4')

# アニメ画を動画に変換
! ffmpeg -r 10 -i images/%6d.png\
               -vcodec libx264 -pix_fmt yuv420p output1.mp4

# スケッチ画を動画に変換
! ffmpeg -r 10 -i out/%6d.png\
               -vcodec libx264 -pix_fmt yuv420p output2.mp4

# アニメ動画とスケッチ動画を連結
! ffmpeg -i output1.mp4 -i output2.mp4 -filter_complex "hstack" output3.mp4

**動画再生**

In [ ]:
# mp4動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output3.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")